In [1]:
from pyspark.sql import SparkSession


In [2]:
spark = SparkSession.builder.appName('rec').getOrCreate()

In [3]:
from pyspark.ml.recommendation import ALS

In [4]:
from pyspark.ml.evaluation import RegressionEvaluator

In [7]:
from email.header import Header


data = spark.read.csv('movielens_ratings.csv', inferSchema= True, header = True )

In [8]:
data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [9]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [10]:
train_data, test_data = data.randomSplit([0.8,0.2])

In [11]:
als = ALS(maxIter = 5, regParam= 0.01, userCol='userId', itemCol='movieId', ratingCol='rating')

In [12]:
model = als.fit(train_data)

In [13]:
predictions = model.transform(test_data)

In [14]:
predictions.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|      0|   3.0|    28| 0.41892394|
|      6|   1.0|    12|  2.7012591|
|      0|   1.0|    22|  0.7884736|
|      5|   3.0|    16|-0.55661416|
|      0|   1.0|     3|  1.0958749|
|      1|   1.0|     3| -1.0635586|
|      1|   1.0|    20| 0.15202492|
|      4|   1.0|    19|  1.9746189|
|      2|   3.0|     9|   1.700001|
|      6|   1.0|     9|  1.1033401|
|      1|   1.0|     4| 0.48928273|
|      1|   1.0|     7|  3.1923232|
|      3|   1.0|     7|  0.4231158|
|      4|   1.0|     7|   2.071145|
|      0|   3.0|    10|  1.1224737|
|      2|   1.0|    25|   2.135292|
|      5|   1.0|    29| 0.11184165|
|      0|   1.0|    11|-0.25941068|
|      6|   1.0|    14|   1.468283|
|      2|   3.0|     0|  5.0395145|
+-------+------+------+-----------+
only showing top 20 rows



In [15]:
evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')


In [16]:
rmse = evaluator.evaluate(predictions)

In [17]:
rmse

2.0343672483120714

In [20]:
single_user = test_data.filter(test_data['userId'] == 11).select(['movieId','userId'])

In [22]:
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|      0|    11|
|     10|    11|
|     19|    11|
|     21|    11|
|     27|    11|
|     35|    11|
|     37|    11|
|     38|    11|
|     43|    11|
|     47|    11|
|     48|    11|
|     67|    11|
|     79|    11|
+-------+------+



In [23]:
recommendations = model.transform(single_user)

In [25]:
recommendations.orderBy('prediction', ascending = False).show()

+-------+------+-----------+
|movieId|userId| prediction|
+-------+------+-----------+
|     27|    11|  3.8852987|
|     35|    11|  1.7895352|
|     21|    11|  1.2826488|
|     47|    11|  1.1194897|
|     37|    11| 0.41765457|
|      0|    11|-0.25941068|
|     38|    11| -0.6170895|
|     48|    11| -0.8102296|
|     19|    11| -1.1944834|
|     79|    11| -1.4787252|
|     67|    11| -2.3249176|
|     10|    11|   -2.46886|
|     43|    11| -2.4789612|
+-------+------+-----------+

